### 70% - 30% split testing.  Added Wins and Losses to the training model

#### Removing the following features ('Playoffs', 'Year', 'Team', 'G', 'MP', 'Conference')

In [31]:
# Libraries being used

import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [32]:
# Loading NBA dataset and viewing the first 20 lines
# Created a Pandas DataFrame called nba_df

nba_df = pd.read_csv('NBA_data.csv')
nba_df.head(10)

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FTA,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS
0,1,Golden State Warriors,1990,82,37,45,West,240.3,42.5,87.9,...,34.9,0.809,11.2,29.1,24.1,9.2,6.0,17.3,24.5,116.3
1,2,Phoenix Suns*,1990,82,54,28,West,242.1,43.2,87.1,...,33.1,0.795,12.8,32.3,25.7,8.1,6.1,15.5,22.3,114.9
2,3,Denver Nuggets*,1990,82,43,39,West,241.5,45.3,97.7,...,26.8,0.789,14.3,30.9,27.7,9.9,4.0,13.9,25.0,114.6
3,4,Portland Trail Blazers*,1990,82,59,23,West,242.4,43.6,92.0,...,33.3,0.743,16.5,31.1,25.4,9.1,4.4,16.5,25.0,114.2
4,5,Orlando Magic,1990,82,18,64,East,241.5,42.2,91.8,...,33.2,0.756,15.9,30.1,24.3,7.5,3.6,17.2,24.1,110.9
5,6,Los Angeles Lakers*,1990,82,63,19,West,242.1,41.9,85.5,...,29.5,0.787,13.4,30.0,27.2,8.0,5.4,15.0,21.2,110.7
6,7,Philadelphia 76ers*,1990,82,53,29,East,241.5,41.9,85.7,...,30.6,0.788,13.5,29.3,23.6,8.4,4.5,14.7,20.7,110.2
7,8,Boston Celtics*,1990,82,52,30,East,240.3,43.5,87.2,...,26.3,0.832,13.0,33.0,29.5,6.6,5.5,15.3,20.9,110.0
8,9,Chicago Bulls*,1990,82,55,27,East,241.8,43.1,86.5,...,26.1,0.778,13.1,27.8,26.5,9.9,4.7,15.2,23.2,109.5
9,10,Indiana Pacers*,1990,82,42,40,East,242.4,41.2,83.0,...,28.5,0.816,11.5,29.1,24.7,6.7,4.3,16.4,24.0,109.3


In [33]:
nba_df.tail()

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FTA,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS
928,26,Oklahoma City Thunder,2021,37,16,21,West,242.0,39.0,87.5,...,20.4,0.739,7.9,36.7,23.1,6.9,4.3,14.4,18.4,106.2
929,27,Houston Rockets,2021,36,11,25,West,240.7,38.6,89.0,...,22.2,0.753,9.1,33.6,22.0,8.1,5.6,14.6,20.3,107.3
930,28,Cleveland Cavaliers,2021,37,14,23,East,243.4,39.4,87.2,...,21.9,0.726,11.2,32.3,23.3,8.4,5.3,15.9,19.3,104.0
931,29,Memphis Grizzlies,2021,34,17,17,West,240.7,42.7,91.8,...,19.7,0.790,11.1,34.4,27.1,9.8,4.4,13.9,19.0,111.8
932,30,San Antonio Spurs,2021,34,19,15,West,241.5,41.5,91.1,...,20.9,0.790,8.9,35.7,25.0,7.4,4.8,11.1,17.4,110.6


### DATA CLEANING


In [34]:
nba_df.shape

(933, 28)

In [35]:
# Creating a 'Playoffs' column for.  Did the team make the playoffs?
# * after teams name identifies whether team made the playoffs or not

nba_df["Playoffs"] = nba_df["Team"].map(lambda x: 1 if "*" in x else 0)

nba_df.head(10)

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
0,1,Golden State Warriors,1990,82,37,45,West,240.3,42.5,87.9,...,0.809,11.2,29.1,24.1,9.2,6.0,17.3,24.5,116.3,0
1,2,Phoenix Suns*,1990,82,54,28,West,242.1,43.2,87.1,...,0.795,12.8,32.3,25.7,8.1,6.1,15.5,22.3,114.9,1
2,3,Denver Nuggets*,1990,82,43,39,West,241.5,45.3,97.7,...,0.789,14.3,30.9,27.7,9.9,4.0,13.9,25.0,114.6,1
3,4,Portland Trail Blazers*,1990,82,59,23,West,242.4,43.6,92.0,...,0.743,16.5,31.1,25.4,9.1,4.4,16.5,25.0,114.2,1
4,5,Orlando Magic,1990,82,18,64,East,241.5,42.2,91.8,...,0.756,15.9,30.1,24.3,7.5,3.6,17.2,24.1,110.9,0
5,6,Los Angeles Lakers*,1990,82,63,19,West,242.1,41.9,85.5,...,0.787,13.4,30.0,27.2,8.0,5.4,15.0,21.2,110.7,1
6,7,Philadelphia 76ers*,1990,82,53,29,East,241.5,41.9,85.7,...,0.788,13.5,29.3,23.6,8.4,4.5,14.7,20.7,110.2,1
7,8,Boston Celtics*,1990,82,52,30,East,240.3,43.5,87.2,...,0.832,13.0,33.0,29.5,6.6,5.5,15.3,20.9,110.0,1
8,9,Chicago Bulls*,1990,82,55,27,East,241.8,43.1,86.5,...,0.778,13.1,27.8,26.5,9.9,4.7,15.2,23.2,109.5,1
9,10,Indiana Pacers*,1990,82,42,40,East,242.4,41.2,83.0,...,0.816,11.5,29.1,24.7,6.7,4.3,16.4,24.0,109.3,1


In [36]:
# Remove * after I identify 'Playoff' teams

nba_df["Team"] = nba_df["Team"].str.replace("*", "")

nba_df.head()

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
0,1,Golden State Warriors,1990,82,37,45,West,240.3,42.5,87.9,...,0.809,11.2,29.1,24.1,9.2,6.0,17.3,24.5,116.3,0
1,2,Phoenix Suns,1990,82,54,28,West,242.1,43.2,87.1,...,0.795,12.8,32.3,25.7,8.1,6.1,15.5,22.3,114.9,1
2,3,Denver Nuggets,1990,82,43,39,West,241.5,45.3,97.7,...,0.789,14.3,30.9,27.7,9.9,4.0,13.9,25.0,114.6,1
3,4,Portland Trail Blazers,1990,82,59,23,West,242.4,43.6,92.0,...,0.743,16.5,31.1,25.4,9.1,4.4,16.5,25.0,114.2,1
4,5,Orlando Magic,1990,82,18,64,East,241.5,42.2,91.8,...,0.756,15.9,30.1,24.3,7.5,3.6,17.2,24.1,110.9,0


In [37]:
nba_df.shape

(933, 29)

In [38]:
nba_df.columns

Index(['RK', 'Team', 'Year', 'G', 'W', 'L', 'Conference', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Playoffs'],
      dtype='object')

In [39]:
# Split 2021 data from 2020 data
# There is no Playoff data for 2021 and that is what we will be predicting

model_df = nba_df.query("Year <= 2020")      # 1991 - 2020 Data
test_df = nba_df.query("Year == 2021")       # 2021 Data


test_df.columns

Index(['RK', 'Team', 'Year', 'G', 'W', 'L', 'Conference', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Playoffs'],
      dtype='object')

In [40]:
test_df.shape

(30, 29)

In [41]:
test_df.head()

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
903,1,Brooklyn Nets,2021,38,25,13,East,243.3,43.8,87.7,...,0.804,8.7,35.6,27.1,6.4,5.3,14.4,19.2,121.1,0
904,2,Los Angeles Clippers,2021,39,25,14,West,240.0,41.9,86.7,...,0.843,9.5,35.0,24.4,7.0,4.6,13.3,19.4,115.2,0
905,3,Milwaukee Bucks,2021,37,23,14,East,240.0,44.4,91.1,...,0.740,10.4,37.6,25.8,8.1,4.9,13.5,17.9,119.2,0
906,4,Philadelphia 76ers,2021,38,26,12,East,242.0,41.8,87.0,...,0.788,10.2,35.6,23.5,8.8,6.2,15.5,19.9,115.3,0
907,5,New Orleans Pelicans,2021,38,16,22,West,241.3,42.9,88.9,...,0.729,12.1,34.8,25.6,7.2,4.1,14.0,17.6,115.2,0


In [42]:
model_df.shape

(903, 29)

In [43]:
model_df.head()

,RK,Team,Year,G,W,L,Conference,MP,FG,FGA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
0,1,Golden State Warriors,1990,82,37,45,West,240.3,42.5,87.9,...,0.809,11.2,29.1,24.1,9.2,6.0,17.3,24.5,116.3,0
1,2,Phoenix Suns,1990,82,54,28,West,242.1,43.2,87.1,...,0.795,12.8,32.3,25.7,8.1,6.1,15.5,22.3,114.9,1
2,3,Denver Nuggets,1990,82,43,39,West,241.5,45.3,97.7,...,0.789,14.3,30.9,27.7,9.9,4.0,13.9,25.0,114.6,1
3,4,Portland Trail Blazers,1990,82,59,23,West,242.4,43.6,92.0,...,0.743,16.5,31.1,25.4,9.1,4.4,16.5,25.0,114.2,1
4,5,Orlando Magic,1990,82,18,64,East,241.5,42.2,91.8,...,0.756,15.9,30.1,24.3,7.5,3.6,17.2,24.1,110.9,0


In [44]:
model_df.columns


Index(['RK', 'Team', 'Year', 'G', 'W', 'L', 'Conference', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Playoffs'],
      dtype='object')

# ML

In [45]:
# Original drop list - ['Playoffs', 'Year', 'Team', 'G', 'MP', 'W', 'L', 'Conference']

y = model_df['Playoffs']  # Playoff column only for 1990 - 2020  (labels)
X = model_df.drop(['Playoffs', 'Year', 'Team', 'G', 'MP', 'Conference', ], axis = 1) # (Features)


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)  # 70% training and 30% test)

X_train.columns

Index(['RK', 'W', 'L', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS'],
      dtype='object')

In [46]:
y_test.value_counts()

1    167
0    104
Name: Playoffs, dtype: int64

In [47]:
X_test.shape

(271, 23)

In [48]:
y_test.shape

(271,)

In [49]:
# Decision Tree

# Create Decision Tree classifer object
dt_model = DecisionTreeClassifier(criterion="entropy")

# Train Decision Tree Classifer
dt_model = dt_model.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = dt_model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

dt_predict = dt_model.predict(X_test)
print(classification_report(y_test, dt_predict))

Accuracy: 86.34686346863468
[[ 98   6]
 [ 31 136]]
              precision    recall  f1-score   support

           0       0.76      0.94      0.84       104
           1       0.96      0.81      0.88       167

    accuracy                           0.86       271
   macro avg       0.86      0.88      0.86       271
weighted avg       0.88      0.86      0.87       271



In [50]:
# Random Forest


#Create a Gaussian Classifier
rfc_model = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
rfc_model.fit(X_train,y_train)

y_pred = rfc_model.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

rfc_predict = rfc_model.predict(X_test)
print(classification_report(y_test, rfc_predict))

Accuracy: 92.25092250922509
[[ 99   5]
 [ 16 151]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       104
           1       0.97      0.90      0.93       167

    accuracy                           0.92       271
   macro avg       0.91      0.93      0.92       271
weighted avg       0.93      0.92      0.92       271



In [51]:
# Naive Bayes
# https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn


#Create a Gaussian Classifier
gnb_model = GaussianNB()

#Train the model using the training sets
gnb_model.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb_model.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

gnb_predict = gnb_model.predict(X_test)
print(classification_report(y_test, gnb_predict))

Accuracy: 87.82287822878229
[[ 97   7]
 [ 26 141]]
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       104
           1       0.95      0.84      0.90       167

    accuracy                           0.88       271
   macro avg       0.87      0.89      0.87       271
weighted avg       0.89      0.88      0.88       271



In [52]:
# Adaboost
# https://www.datacamp.com/community/tutorials/adaboost-classifier-python


# Create adaboost classifer object
abc_model = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
abc_model = abc_model.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = abc_model.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

abc_predict = abc_model.predict(X_test)
print(classification_report(y_test, abc_predict))

Accuracy: 91.5129151291513
[[ 95   9]
 [ 14 153]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       104
           1       0.94      0.92      0.93       167

    accuracy                           0.92       271
   macro avg       0.91      0.91      0.91       271
weighted avg       0.92      0.92      0.92       271



In [53]:
# KNN


knn_model = KNeighborsClassifier()

knn_model.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn_model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

knn_predict = knn_model.predict(X_test)
print(classification_report(y_test, abc_predict))

Accuracy: 92.619926199262
[[ 93  11]
 [  9 158]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       104
           1       0.94      0.92      0.93       167

    accuracy                           0.92       271
   macro avg       0.91      0.91      0.91       271
weighted avg       0.92      0.92      0.92       271



In [54]:
# Logistic Regression


log_model = LogisticRegression()

log_model.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = log_model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) * 100)

# Confusion Matrix - Shows True Positive, True negative, False Positive, False Negative
print(metrics.confusion_matrix(y_test, y_pred))

log_predict = log_model.predict(X_test)
print(classification_report(y_test, abc_predict))


Accuracy: 92.25092250922509
[[ 98   6]
 [ 15 152]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       104
           1       0.94      0.92      0.93       167

    accuracy                           0.92       271
   macro avg       0.91      0.91      0.91       271
weighted avg       0.92      0.92      0.92       271



### Random Forest 

Accuracy = 94%

True Positives = 153
True Negatives = 101
False Positives = 3
False Negatives = 14

Precision = 94%
Recall = 94%
F1 = 94%

In [55]:
# Original drop list - ['Playoffs', 'Year', 'Team', 'G', 'MP', 'W', 'L', 'Conference']

test_df2 = test_df.drop(['Playoffs', 'Year', 'Team', 'G', 'MP', 'Conference'], axis = 1) 

y_predict =rfc_model.predict(test_df2)
print(y_predict)

[1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0]


In [56]:
predict_data = test_df[['Team', 'Year', 'Playoffs', "Conference"]]
predict_data['Playoffs'] = y_predict
predict_data

,Team,Year,Playoffs,Conference
903,Brooklyn Nets,2021,1,East
904,Los Angeles Clippers,2021,1,West
905,Milwaukee Bucks,2021,1,East
906,Philadelphia 76ers,2021,1,East
907,New Orleans Pelicans,2021,0,West
908,Utah Jazz,2021,1,West
909,Golden State Warriors,2021,0,West
910,Denver Nuggets,2021,1,West
911,Sacramento Kings,2021,1,West
912,Los Angeles Lakers,2021,0,West


In [57]:
predict_2021 = predict_data.query("Playoffs == 1")

predict_2021.sort_values('Conference')


,Team,Year,Playoffs,Conference
903,Brooklyn Nets,2021,1,East
905,Milwaukee Bucks,2021,1,East
906,Philadelphia 76ers,2021,1,East
913,Toronto Raptors,2021,1,East
914,Atlanta Hawks,2021,1,East
918,Chicago Bulls,2021,1,East
922,Indiana Pacers,2021,1,East
904,Los Angeles Clippers,2021,1,West
908,Utah Jazz,2021,1,West
910,Denver Nuggets,2021,1,West


In [58]:
predict_2021.shape

(14, 4)

### Predicted 14 teams.  (16 go to the playoffs)
#### 6 East and 8 West

### Adaboost

In [68]:
test_df3 = test_df.drop(['Playoffs', 'Year', 'Team', 'G', 'MP', 'Conference'], axis = 1) 

y_predict_3 =abc_model.predict(test_df3)
print(y_predict)

[0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0]


In [69]:
predict_data_3 = test_df[['Team', 'Year', 'Playoffs', "Conference"]]
predict_data['Playoffs'] = y_predict_3
predict_data_3

,Team,Year,Playoffs,Conference
903,Brooklyn Nets,2021,0,East
904,Los Angeles Clippers,2021,0,West
905,Milwaukee Bucks,2021,0,East
906,Philadelphia 76ers,2021,0,East
907,New Orleans Pelicans,2021,0,West
908,Utah Jazz,2021,0,West
909,Golden State Warriors,2021,0,West
910,Denver Nuggets,2021,0,West
911,Sacramento Kings,2021,0,West
912,Los Angeles Lakers,2021,0,West


In [70]:
predict_2021_3 = predict_data.query("Playoffs == 1")

predict_2021_3.sort_values('Conference')


,Team,Year,Playoffs,Conference
920,Miami Heat,2021,1,East
905,Milwaukee Bucks,2021,1,East
927,Orlando Magic,2021,1,East
913,Toronto Raptors,2021,1,East
914,Atlanta Hawks,2021,1,East
926,New York Knicks,2021,1,East
917,Washington Wizards,2021,1,East
925,Detroit Pistons,2021,1,East
921,Charlotte Hornets,2021,1,East
928,Oklahoma City Thunder,2021,1,West


In [66]:
predict_2021_2.shape

(19, 4)

### Predicted 19 teams.  (16 go to the playoffs)
#### 9 East and 10 West

### Logistic Regression

In [71]:
test_df4 = test_df.drop(['Playoffs', 'Year', 'Team', 'G', 'MP', 'Conference'], axis = 1) 

y_predict_4 =log_model.predict(test_df4)
print(y_predict)

[0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0]


In [72]:
predict_data_4 = test_df[['Team', 'Year', 'Playoffs', "Conference"]]
predict_data['Playoffs'] = y_predict_4
predict_data_4

,Team,Year,Playoffs,Conference
903,Brooklyn Nets,2021,0,East
904,Los Angeles Clippers,2021,0,West
905,Milwaukee Bucks,2021,0,East
906,Philadelphia 76ers,2021,0,East
907,New Orleans Pelicans,2021,0,West
908,Utah Jazz,2021,0,West
909,Golden State Warriors,2021,0,West
910,Denver Nuggets,2021,0,West
911,Sacramento Kings,2021,0,West
912,Los Angeles Lakers,2021,0,West


In [77]:
predict_2021_4 = predict_data.query("Playoffs == 1")

predict_2021_4.sort_values('Conference')

,Team,Year,Playoffs,Conference
903,Brooklyn Nets,2021,1,East
905,Milwaukee Bucks,2021,1,East
906,Philadelphia 76ers,2021,1,East
913,Toronto Raptors,2021,1,East
915,Boston Celtics,2021,1,East
920,Miami Heat,2021,1,East
921,Charlotte Hornets,2021,1,East
904,Los Angeles Clippers,2021,1,West
908,Utah Jazz,2021,1,West
910,Denver Nuggets,2021,1,West


In [78]:
predict_2021_4.shape

(16, 4)

### Predicted 16 teams.  (16 go to the playoffs)
#### 7 East and 9 West